In [1]:
%load_ext dotenv
%dotenv keys.env

from requests import Request,Session,Response
import time
import datetime
import json
import logging
import hmac
import hashlib
import os
from dotenv import load_dotenv
import urllib
from typing import Optional, Dict, Any, List

In [2]:
os.getenv('FTX_HUNTER_KEY')


'IMnGcWGSpSc4KgMM2kLnSmu8lWo9dFBaxcDIV0Oz'

In [3]:
from colorama import Fore, Back, Style, init
logging.basicConfig(level=logging.INFO, format=(
    Fore.BLUE + '[+] ' + Style.RESET_ALL + '%(message)s '))
logger = logging.getLogger("Logger")

In [59]:
class FtxClient:
    _ENDPOINT = 'https://ftx.com/api/'
    markets = {
        "ATOM":"ATOM-PERP",
        "XTZ":"XTZ-PERP",
        "BTC":"BTC-PERP",
        "ETH":"ETH-PERP"
        
    }
    def __init__(self,subaccount_name = None) -> None:
        self._session = Session()
        self._api_key = os.getenv('FTX_HUNTER_KEY')
        self._api_secret = os.getenv('FTX_HUNTER_SECRET')
        self._subaccount_name = subaccount_name
    
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, json=params)
    def _delete(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('DELETE', path, json=params)
    def _request(self,method:str,path:str, **kwargs) -> Any:
        request = Request(method,self._ENDPOINT + path, **kwargs)
        # Apply hash to keys
        self._sign_request(request)
        # Send the request, similar to req.get() or req.post()
        response = self._session.send(request.prepare())
        # Clean up response
        result = self._process_response(response)
        logger.info(f'{result}')
    def _sign_request(self,request:Request) -> None:
        ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        
        if prepared.body:
            signature_payload += prepared.body
        logger.info(f'{signature_payload}')
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    def get_account_info(self) -> dict:
        return self._get(f'account')
    def list_markets(self) -> List[dict]:
        return self._get('markets')
    def place_order(self, market: str, side: str, price: float, size: float, type: str = 'limit',
                    reduce_only: bool = False, ioc: bool = False, post_only: bool = False,
                    clientId: str = None) -> dict:
        return self._post('orders', {'market': market,
                                     'side': side,
                                     'price': price,
                                     'size': size,
                                     'type': type,
                                     'reduceOnly': reduce_only,
                                     'ioc': ioc,
                                     'postOnly': post_only,
                                     'clientId': clientId,
                                     })
    def place_conditional_order(
        self, market: str, side: str, size: float, type: str,
        limit_price: float = None, reduce_only: bool = False, cancel: bool = True,
        trigger_price: float = None, trail_value: float = None, clientId: str = None) -> dict:
        """
        To send a Stop Market order, set type='stop' and supply a trigger_price
        To send a Stop Limit order, also supply a limit_price
        To send a Take Profit Market order, set type='trailing_stop' and supply a trigger_price
        To send a Trailing Stop order, set type='trailing_stop' and supply a trail_value
        """
        assert type in ('stop', 'take_profit', 'trailing_stop')
        assert type not in ('stop', 'take_profit') or trigger_price is not None, \
            'Need trigger prices for stop losses and take profits'
        assert type not in ('trailing_stop',) or (trigger_price is None and trail_value is not None), \
            'Trailing stops need a trail value and cannot take a trigger price'

        return self._post('conditional_orders',
                          {'market': market, 'side': side, 'triggerPrice': trigger_price,
                           'size': size, 'reduceOnly': reduce_only, 'type': type,
                           'cancelLimitOnTrigger': cancel, 'orderPrice': limit_price, 'clientId': clientId})
    
    def cancel_order(self, order_id: str) -> dict:
        return self._delete(f'orders/{order_id}')

    def cancel_orders(self, market_name: str = None, conditional_orders: bool = False,
                      limit_orders: bool = False) -> dict:
        return self._delete(f'orders', {'market': market_name,
                                        'conditionalOrdersOnly': conditional_orders,
                                        'limitOrdersOnly': limit_orders,
                                        })
    def get_open_orders(self, market: str = None) -> List[dict]:
        return self._get(f'orders', {'market': market})

In [60]:
ftx = FtxClient(subaccount_name = 'hunter-api')

In [58]:
ftx.get_account_info()

[+] b'1595388491448GET/api/account' 
[+] {'backstopProvider': False, 'chargeInterestOnNegativeUsd': False, 'collateral': 96.42603021051, 'freeCollateral': 93.65585789742, 'initialMarginRequirement': 0.1, 'leverage': 10.0, 'liquidating': False, 'maintenanceMarginRequirement': 0.03, 'makerFee': 0.00019, 'marginFraction': None, 'openMarginFraction': 31.558754458170704, 'positionLimit': None, 'positionLimitUsed': None, 'positions': [{'collateralUsed': 0.29771, 'cost': 0.0, 'entryPrice': None, 'estimatedLiquidationPrice': None, 'future': 'XTZ-PERP', 'initialMarginRequirement': 0.1, 'longOrderSize': 1.0, 'maintenanceMarginRequirement': 0.03, 'netSize': 0.0, 'openSize': 1.0, 'realizedPnl': 0.0, 'shortOrderSize': 0.0, 'side': 'buy', 'size': 0.0, 'unrealizedPnl': 0.0}], 'spotLendingEnabled': False, 'spotMarginEnabled': False, 'takerFee': 0.000665, 'totalAccountValue': 96.42603021051, 'totalPositionSize': 0.0, 'useFttCollateral': True, 'username': 'deerhunter001@protonmail.com/hunter-api'} 


In [61]:
ftx.get_open_orders(ftx.markets.get('XTZ'))

[+] b'1595388614624GET/api/orders?market=XTZ-PERP' 
[+] [{'avgFillPrice': None, 'clientId': None, 'createdAt': '2020-07-22T03:17:04.890288+00:00', 'filledSize': 0.0, 'future': 'XTZ-PERP', 'id': 6754573045, 'ioc': False, 'liquidation': False, 'market': 'XTZ-PERP', 'postOnly': False, 'price': 1.89, 'reduceOnly': False, 'remainingSize': 1.0, 'side': 'buy', 'size': 1.0, 'status': 'open', 'type': 'limit'}] 


In [63]:
ftx.cancel_orders(market_name=ftx.markets.get('XTZ'))

[+] b'1595388682115DELETE/api/orders{"market": "XTZ-PERP", "conditionalOrdersOnly": false, "limitOrdersOnly": false}' 
[+] Orders queued for cancellation 


In [50]:
#placing limit order
market = ftx.markets.get('XTZ')
side = "buy"
price = "1.89"
size = 1
type = "limit"
clientId = f'{market}_5'

In [52]:
ftx.place_order(market=market, side=side, price=price, size=size, type=type)

[+] b'1595387825536POST/api/orders{"market": "XTZ-PERP", "side": "buy", "price": "1.89", "size": 1, "type": "limit", "reduceOnly": false, "ioc": false, "postOnly": false, "clientId": null}' 
[+] {'avgFillPrice': None, 'clientId': None, 'createdAt': '2020-07-22T03:17:04.890288+00:00', 'filledSize': 0.0, 'future': 'XTZ-PERP', 'id': 6754573045, 'ioc': False, 'liquidation': False, 'market': 'XTZ-PERP', 'postOnly': False, 'price': 1.89, 'reduceOnly': False, 'remainingSize': 1.0, 'side': 'buy', 'size': 1.0, 'status': 'new', 'type': 'limit'} 


In [53]:
#placing stoploss - STOP MARKET
typeStop = 'stop'
market=ftx.markets.get('XTZ')
side = "sell"
trigger_price_stoploss = 1.75
sl_clientId = 'stoploss_order2'
sizeSL = 1

#placing target point - TAKE PROFIT MARKET
typeTP = "take_profit"
market = ftx.markets.get('XTZ')
side = "sell"
trigger_price_tp = 4.5
tp_clientId = 'targetpoint_order4'
sizeTP = 1

In [ ]:
ftx.place_order(market=market, side=side, price=price, size=size, type=type)


In [54]:
ftx.place_conditional_order(market=market,side=side,size=sizeSL,trigger_price = trigger_price_stoploss,type = typeStop,clientId=sl_clientId)
#


[+] b'1595387860919POST/api/conditional_orders{"market": "XTZ-PERP", "side": "sell", "triggerPrice": 1.75, "size": 1, "reduceOnly": false, "type": "stop", "cancelLimitOnTrigger": true, "orderPrice": null, "clientId": null}' 
[+] {'cancelReason': None, 'cancelledAt': None, 'createdAt': '2020-07-22T03:17:40.249300+00:00', 'error': None, 'future': 'XTZ-PERP', 'id': 5774886, 'market': 'XTZ-PERP', 'orderId': None, 'orderPrice': None, 'orderType': 'market', 'reduceOnly': False, 'retryUntilFilled': True, 'side': 'sell', 'size': 1.0, 'status': 'open', 'trailStart': None, 'trailValue': None, 'triggerPrice': 1.75, 'triggeredAt': None, 'type': 'stop'} 


In [55]:
ftx.place_conditional_order(market=market,side=side,size=sizeTP,trigger_price = trigger_price_tp,type = typeTP)

[+] b'1595387861758POST/api/conditional_orders{"market": "XTZ-PERP", "side": "sell", "triggerPrice": 4.5, "size": 1, "reduceOnly": false, "type": "take_profit", "cancelLimitOnTrigger": true, "orderPrice": null, "clientId": null}' 
[+] {'cancelReason': None, 'cancelledAt': None, 'createdAt': '2020-07-22T03:17:41.852798+00:00', 'error': None, 'future': 'XTZ-PERP', 'id': 5774892, 'market': 'XTZ-PERP', 'orderId': None, 'orderPrice': None, 'orderType': 'market', 'reduceOnly': False, 'retryUntilFilled': True, 'side': 'sell', 'size': 1.0, 'status': 'open', 'trailStart': None, 'trailValue': None, 'triggerPrice': 4.5, 'triggeredAt': None, 'type': 'take_profit'} 
